<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Asset Management

### Risk Measures

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

### The use of the "Python 3.10, Numpy 1.26.4" kernel is recommended.

### The use of the "Python 3.10, Numpy 1.26.4" kernel is recommended.

## Coherent Risk Measures

Topics of interest include:

* Risk Measures
 * Volatility
 * Standard Deviation-Based
 * Value-at-Risk
 * Expected Shortfall
* Characteristics
 * Subadditivity
 * Homogeneity
 * Monotonicity
 * Translation Invariance

## Real Data

**_Historical end-of-day financial time series data._**

Data retrieved vom Refinitiv Eikon for the German DAX 30 constituents.

The data sets:

    http://hilpisch.com/dax_eikon_eod_data.csv
    http://hilpisch.com/dax_eikon_mc_data.csv

## Imports and Data

In [ ]:
import math
import cufflinks
import eikon as ek
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn-v0_8')
cufflinks.set_config_file(offline=True)
np.set_printoptions(suppress=True, precision=4)
pd.options.display.float_format = '{:.5f}'.format
%config InlineBackend.figure_format = 'svg'

In [ ]:
url = 'http://hilpisch.com/dax_eikon_eod_data.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True)

In [ ]:
raw.columns

In [ ]:
raw.iloc[:, :10].normalize().iplot()

## Volatility

In [ ]:
symbols = raw.columns[:3]
symbols

In [ ]:
data = raw[symbols]

In [ ]:
rets = np.log(data / data.shift(1))

In [ ]:
def portfolio_return(weights, rets=rets):
    return np.dot(weights, rets.mean()) * 252

In [ ]:
def portfolio_variance(weights, rets=rets):
    return np.dot(weights, np.dot(rets.cov(), weights)) * 252

In [ ]:
def portfolio_volatility(weights, rets=rets):
    return math.sqrt(portfolio_variance(weights, rets))

### Subadditivity

In [ ]:
w = np.array(len(symbols) * [1 / len(symbols)])
w

In [ ]:
np.dot(rets.std() * math.sqrt(252), w)

In [ ]:
portfolio_volatility(w)

### Homogeneity 

In [ ]:
lamb = 0.5

In [ ]:
portfolio_volatility(lamb * w)

In [ ]:
lamb * portfolio_volatility(w)

In [ ]:
lamb = 2

In [ ]:
portfolio_volatility(lamb * w)

In [ ]:
lamb * portfolio_volatility(w)

### Monotonicity

In [ ]:
portfolio_volatility(w)

In [ ]:
portfolio_volatility(w, rets + 0.001)  # volatility not changed

In [ ]:
portfolio_volatility(w, rets * 1.01)  # volatility increases (due to scaling)

In [ ]:
(rets * 1.01 > rets).tail(5)  # assumption not given

In [ ]:
rnd = np.random.random(rets.shape) * 0.02
rnd

In [ ]:
rnd = np.random.normal(0.001, 0.0001, rets.shape) 
rnd

In [ ]:
(rnd > 0).sum()

In [ ]:
rets.shape[0] * rets.shape[1]

In [ ]:
portfolio_volatility(w, rets + rnd)  # volatility increases due to added randomness

In [ ]:
(rets + rnd > rets).tail(8)  # assumption given

### Translation Invariance

In [ ]:
m = 0.1

In [ ]:
rets_ = rets.copy()

In [ ]:
rets_['cash'] = 0

In [ ]:
w_ = np.insert(w, len(w), 0)

In [ ]:
w_

In [ ]:
portfolio_volatility(w)

In [ ]:
portfolio_volatility(w_, rets_)

In [ ]:
wm = w - m / len(w)
wm = np.insert(wm, 3, m)
wm

In [ ]:
portfolio_volatility(wm, rets_)

In [ ]:
red = portfolio_volatility(w) - portfolio_volatility(wm, rets_)
red

In [ ]:
red / portfolio_volatility(w)

## Standard Deviation-Based Risk Measure

In [ ]:
def standard_deviation_mu(weights, rets=rets, c=1):
    return (c * portfolio_volatility(weights, rets) -
            portfolio_return(weights, rets))

### Subadditivity

In [ ]:
w = np.array(len(symbols) * [1 / len(symbols)])
w

In [ ]:
np.dot(rets.std() * math.sqrt(252) - rets.mean() * 252, w)

In [ ]:
standard_deviation_mu(w)

### Homogeneity 

In [ ]:
lamb = 0.5

In [ ]:
standard_deviation_mu(lamb * w)

In [ ]:
lamb * standard_deviation_mu(w)

In [ ]:
lamb = 2

In [ ]:
standard_deviation_mu(lamb * w)

In [ ]:
lamb * standard_deviation_mu(w)

### Monotonicity

In [ ]:
standard_deviation_mu(w)

In [ ]:
rnd.mean()

In [ ]:
standard_deviation_mu(w, rets + rnd)

### Translation Invariance

In [ ]:
standard_deviation_mu(w)

In [ ]:
standard_deviation_mu(w_, rets_)

In [ ]:
standard_deviation_mu(wm, rets_)

In [ ]:
red = standard_deviation_mu(w) - standard_deviation_mu(wm, rets_)
red

In [ ]:
red / standard_deviation_mu(w)

## Value-at-Risk

In [ ]:
import scipy.stats as scs

In [ ]:
def value_at_risk(weights, rets=rets, percs=None, out=True):
    r = np.dot(rets, weights)
    if percs is None:
        percs = [0.01, 0.1, 1., 2.5, 5.0, 10.0]
    var = scs.scoreatpercentile(r, percs)
    if out:
        print('%16s %16s' % ('Confidence Level', 'Value-at-Risk'))
        print(33 * '-')
    v = list()
    for pair in zip(percs, var):
        if out:
            print('%16.2f %16.5f' % (100 - pair[0], -pair[1]))
        v.append((100 - pair[0], -pair[1]))
    df = pd.DataFrame(v, columns=['conf', 'value_at_risk'])
    df.set_index('conf', inplace=True)
    return df

In [ ]:
first = True
for col in rets.columns:
    if first:
        var = value_at_risk(1, rets.loc[:, col], out=False)
        var.columns = [col,]
        first = False
    else:
        var[col] = value_at_risk(1, rets.loc[:, col], out=False)['value_at_risk']

In [ ]:
var

### Subadditivity

In [ ]:
w

In [ ]:
var['value_at_risk_w'] = np.dot(var, w)

In [ ]:
var

In [ ]:
value_at_risk(w, out=False)['value_at_risk']

In [ ]:
var['value_at_risk_w'] - value_at_risk(w, out=False)['value_at_risk']

### Homogeneity 

In [ ]:
lamb = 0.5

In [ ]:
var = value_at_risk(lamb * w, out=False)

In [ ]:
var

In [ ]:
lamb * value_at_risk(w, out=False)

In [ ]:
lamb = 2

In [ ]:
var = value_at_risk(lamb * w, out=False)

In [ ]:
var

In [ ]:
lamb * value_at_risk(w, out=False)

### Monotonicity

In [ ]:
v0 = value_at_risk(w)

In [ ]:
v1 = value_at_risk(w, rets + rnd)

In [ ]:
v0 - v1

### Translation Invariance

In [ ]:
m = 0.1

In [ ]:
var = value_at_risk(w)

In [ ]:
var = value_at_risk(w_, rets_)

In [ ]:
varm = value_at_risk(wm, rets_)

In [ ]:
red = var - varm
red

In [ ]:
red / var

## Expected Shortfall

In [ ]:
def expected_shortfall(weights, rets=rets, percs=None, out=True):
    r = np.dot(rets, weights)
    if percs is None:
        percs = [0.01, 0.1, 1., 2.5, 5.0, 10.0]
    var = scs.scoreatpercentile(r, percs)
    if out:
        print('%16s %16s %16s' % ('Confidence Level', 'Value-at-Risk',
                                  'Exp. Shortfall'))
        print(50 * '-')
    v = list()
    for pair in zip(percs, var):
        es = r[r < pair[1]].mean()
        if out:
            print('%16.2f %16.5f %16.5f' % (100 - pair[0], -pair[1], -es))
        v.append((100 - pair[0], -pair[1], -es))
    df = pd.DataFrame(v, columns=['conf', 'value_at_risk', 'expected_shortfall'])
    df.set_index('conf', inplace=True)
    return df

In [ ]:
first = True
for col in rets.columns:
    if first:
        es = expected_shortfall(1, rets.loc[:, col], out=False)
        del es['value_at_risk']
        es.columns = [col,]
        first = False
    else:
        es[col] = expected_shortfall(1, rets.loc[:, col], out=False)['expected_shortfall']

In [ ]:
es

### Subadditivity

In [ ]:
es['expected_shortfall_w'] = np.dot(es, w)

In [ ]:
es

In [ ]:
esw = expected_shortfall(w)

In [ ]:
es['expected_shortfall_w'] - esw['expected_shortfall']

### Homogeneity 

In [ ]:
lamb = 0.5

In [ ]:
expected_shortfall(lamb * w, out=False)

In [ ]:
lamb * expected_shortfall(w, out=False)

In [ ]:
lamb = 2

In [ ]:
expected_shortfall(lamb * w, out=False)

In [ ]:
lamb * expected_shortfall(w, out=False)

### Monotonicity

In [ ]:
es0 = expected_shortfall(w)

In [ ]:
es1 = expected_shortfall(w, rets + rnd)

In [ ]:
es0 - es1

### Translation Invariance

In [ ]:
m = 0.1

In [ ]:
es = expected_shortfall(w)

In [ ]:
es = expected_shortfall(w_, rets_)

In [ ]:
esm = expected_shortfall(wm, rets_)

In [ ]:
red = es - esm 
red

In [ ]:
red / es

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>